In [26]:
import pandas as pd
import requests
df_agri=pd.read_csv("./Cleaned CSVs/Bajra-2015-2019.csv")
df_agri['Price Date'] = pd.to_datetime(df_agri['Price Date'])
df_agri.head()

,District Name,Market Name,Commodity,Variety,Grade,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Price Date,Day Of Week
0,Ariyalur,Ariyalur Market,Bajra(Pearl Millet/Cumbu),Other,FAQ,1371.0,1379.0,1372.0,2017-11-22,2
1,Ariyalur,Ariyalur Market,Bajra(Pearl Millet/Cumbu),Other,FAQ,1371.0,1480.0,1480.0,2016-01-18,0
2,Ariyalur,Ariyalur Market,Bajra(Pearl Millet/Cumbu),Other,FAQ,1700.0,1739.0,1728.0,2017-08-29,1
3,Ariyalur,Ariyalur Market,Bajra(Pearl Millet/Cumbu),Other,FAQ,1960.0,2000.0,2000.0,2015-03-10,1
4,Cuddalore,Cuddalore,Bajra(Pearl Millet/Cumbu),Bold,FAQ,1519.0,1519.0,1519.0,2016-08-29,0


In [27]:
CROP_LOOKBACK_DAYS = {
    'Bajra(Pearl Millet/Cumbu)': 90,
    
    'Maize': 120,
    'Paddy': 135
}

In [28]:
DISTRICT_COORDINATES = {
    'Ariyalur' : {'lat' : 11.1404, 'lon' : 79.0745},
    'Cuddalore' : {'lat' : 11.7480, 'lon' : 79.7714},
    'Coimbatore': {'lat': 11.0168, 'lon': 76.9558},
    'Salem' : {'lat' : 11.6643, 'lon' : 78.1460},
    'Vellore' : {'lat' : 12.9236, 'lon' : 79.1331},
    'Erode' : {'lat' : 11.3410, 'lon' : 77.7172},
    'Theni' : {'lat' : 10.0079, 'lon' : 77.4735},

    'Tiruchirappalli': {'lat': 10.7905, 'lon': 78.7047},
    'Madurai': {'lat': 9.9252, 'lon': 78.1198}
}

In [29]:
districts = {k: v for k, v in DISTRICT_COORDINATES.items() if k in df_agri['District Name'].unique()}
lookbacks = {k: v for k, v in CROP_LOOKBACK_DAYS.items() if k in df_agri['Commodity'].unique()}

print(f"Districts being processed for this file: {list(districts.keys())}")
print(f"Lookbacks being processed for this file: {lookbacks}")

Districts being processed for this file: ['Ariyalur', 'Cuddalore', 'Coimbatore', 'Salem', 'Vellore', 'Erode', 'Theni']
Lookbacks being processed for this file: {'Bajra(Pearl Millet/Cumbu)': 90}


In [35]:
all_weather_data = []
unique_lookbacks = set(lookbacks.values())
start_date = df_agri['Price Date'].min() - pd.Timedelta(days=max(unique_lookbacks, default=0))
end_date = df_agri['Price Date'].max()

print(unique_lookbacks)

{90}


In [38]:
for district, coords in districts.items():
    print(f"Fetching weather data for {df_agri['Commodity'].unique()} in {district}...")
    
    weather_url = f"https://archive-api.open-meteo.com/v1/archive?latitude={coords['lat']}&longitude={coords['lon']}&start_date={start_date.strftime('%Y-%m-%d')}&end_date={end_date.strftime('%Y-%m-%d')}&daily=temperature_2m_mean,precipitation_sum"
    response = requests.get(weather_url)
    
    if response.status_code == 200:
        df_weather = pd.DataFrame(response.json()['daily'])
        df_weather['Price Date'] = pd.to_datetime(df_weather['time'])
        df_weather['District Name'] = district

        for days in unique_lookbacks:
            df_weather[f'temp_mean_{days}d'] = df_weather['temperature_2m_mean'].rolling(window=days, min_periods=1).mean()
            df_weather[f'precip_sum_{days}d'] = df_weather['precipitation_sum'].rolling(window=days, min_periods=1).sum()
        
        all_weather_data.append(df_weather)

df_weather_master = pd.concat(all_weather_data)

Fetching weather data for ['Bajra(Pearl Millet/Cumbu)'] in Ariyalur...
Fetching weather data for ['Bajra(Pearl Millet/Cumbu)'] in Cuddalore...
Fetching weather data for ['Bajra(Pearl Millet/Cumbu)'] in Coimbatore...
Fetching weather data for ['Bajra(Pearl Millet/Cumbu)'] in Salem...
Fetching weather data for ['Bajra(Pearl Millet/Cumbu)'] in Vellore...
Fetching weather data for ['Bajra(Pearl Millet/Cumbu)'] in Erode...
Fetching weather data for ['Bajra(Pearl Millet/Cumbu)'] in Theni...


In [40]:
print(df_weather_master)

            time  temperature_2m_mean  precipitation_sum Price Date  \
0     2014-10-16                 26.3                7.6 2014-10-16   
1     2014-10-17                 25.0               15.0 2014-10-17   
2     2014-10-18                 24.1               22.8 2014-10-18   
3     2014-10-19                 23.5               29.9 2014-10-19   
4     2014-10-20                 23.6               39.2 2014-10-20   
...          ...                  ...                ...        ...   
1521  2018-12-15                 24.1                0.0 2018-12-15   
1522  2018-12-16                 24.1                0.0 2018-12-16   
1523  2018-12-17                 24.4                0.1 2018-12-17   
1524  2018-12-18                 24.4               13.5 2018-12-18   
1525  2018-12-19                 23.7               12.5 2018-12-19   

     District Name  temp_mean_90d  precip_sum_90d  
0         Ariyalur      26.300000             7.6  
1         Ariyalur      25.650000          

In [43]:
df_merged = pd.merge(df_agri, df_weather_master, on=['District Name', 'Price Date'], how='left')
print(df_merged)

    District Name      Market Name                  Commodity Variety Grade  \
0        Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
1        Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
2        Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
3        Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
4       Cuddalore        Cuddalore  Bajra(Pearl Millet/Cumbu)    Bold   FAQ   
..            ...              ...                        ...     ...   ...   
276    Coimbatore        Thiruppur  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
277    Coimbatore        Thiruppur  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
278    Coimbatore        Thiruppur  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
279    Coimbatore        Thiruppur  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
280     Cuddalore    Virudhachalam  Bajra(Pearl Millet/Cumbu)   Other   FAQ   

     Min Price (Rs./Quintal)  Max Price (Rs./Quinta

In [53]:
def get_correct_lookback_value(row, feature_prefix, lookback_dict):
    commodity = row['Commodity']
    # ** NEW: Check if the commodity exists in our lookback dictionary **
    if commodity in lookback_dict:
        lookback_days = lookback_dict[commodity]
        column_name = f"{feature_prefix}_{lookback_days}d"
        return row[column_name]
    return None

df_merged['lookback_temp_mean'] = df_merged.apply(get_correct_lookback_value, feature_prefix='temp_mean', lookback_dict=lookbacks, axis=1)
df_merged['lookback_precip_sum'] = df_merged.apply(get_correct_lookback_value, feature_prefix='precip_sum', lookback_dict=lookbacks, axis=1)


In [45]:
print(df_merged.head())

  District Name      Market Name                  Commodity Variety Grade  \
0      Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
1      Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
2      Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
3      Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other   FAQ   
4     Cuddalore        Cuddalore  Bajra(Pearl Millet/Cumbu)    Bold   FAQ   

   Min Price (Rs./Quintal)  Max Price (Rs./Quintal)  \
0                   1371.0                   1379.0   
1                   1371.0                   1480.0   
2                   1700.0                   1739.0   
3                   1960.0                   2000.0   
4                   1519.0                   1519.0   

   Modal Price (Rs./Quintal) Price Date  Day Of Week        time  \
0                     1372.0 2017-11-22            2  2017-11-22   
1                     1480.0 2016-01-18            0  2016-01-18   
2 

In [59]:
final_columns = [
    'District Name', 'Market Name', 'Commodity', 'Variety', 'Grade', 'Min Price (Rs./Quintal)', 
    'Max Price (Rs./Quintal)', 'Modal Price (Rs./Quintal)', 'Price Date', 'Day Of Week',
    'lookback_temp_mean', 'lookback_precip_sum'
]

df_final = df_merged[[col for col in final_columns if col in df_merged.columns]].copy()


In [61]:
df_final.to_csv('./Weather_Merged_CSVs/Bajra-2015-2019.csv', index=False)